단어 클라우드 생성위한 카운트를 목적

### (필요할 때만) 설치

In [8]:
!pip install -q gradio wordcloud kiwipiepy
!apt-get -y install -qq fonts-nanum


### 설정 & 토크나이저

In [ ]:
from collections import Counter
from pathlib import Path
import os, re, unicodedata

TARGETS = ["우울증","불안장애","중독","정상군"]

# 한글 폰트 경로 자동 선택
FONT_CANDIDATES = [
    "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    "/System/Library/Fonts/AppleSDGothicNeo.ttc",
    "C:/Windows/Fonts/malgun.ttf",
]
FONT_PATH = next((p for p in FONT_CANDIDATES if os.path.exists(p)), None)

# 토크나이저: 명사 위주(권장) → 실패 시 심플 토크나이저
USE_KIWI = True
try:
    if USE_KIWI:
        from kiwipiepy import Kiwi
        kiwi = Kiwi()
        def tokenize(text: str):
            toks = []
            for (form, tag, _, _ ) in kiwi.analyze(text, top_n=1)[0][0]:
                if tag.startswith("N","VV"):  # 명사류, 동사
                    toks.append(form)
            return toks
    else:
        raise ImportError
except Exception:
    def tokenize(text: str):
        text = unicodedata.normalize("NFKC", str(text))
        return re.findall(r"[가-힣]{2,}|[A-Za-z]{2,}|[0-9]{2,}", text)

# 불용어 예시(원하면 추가/수정)
STOPWORDS = set("""
그리고 그러나 그런데 그래서 또한 또는 정말 매우 너무 그냥 어떤 이런 저런 그런 정도 조금 거의
상담 병원 증상 치료 검사 결과 문제 상태 환자 선생님 경험 이야기 생각 느낌 오늘 지금 최근 번호 문의
""".split())


### “모델 점수” 연동 훅

✅ 여기만 당신의 모델 호출로 바꿔주면 됩니다.
반환형: {"우울증": float, "불안장애": float, "중독": float, "정상군": float} (합이 1에 가까우면 베스트)

In [10]:
def get_scores(text: str) -> dict:
    """
    TODO: 당신의 '문장을 받으면 점수' 모델로 교체하세요.
    예시(Transformers 분류 모델일 때):
        inputs = tok(text, return_tensors="pt", truncation=True).to(model.device)
        with torch.no_grad():
            prob = model(**inputs).logits.softmax(-1).cpu().numpy()[0]
        # id2label를 한국어 라벨로 매핑 후 dict 구성
    """
    # --- 데모용(무작위) : 반드시 교체하세요 ---
    import random
    xs = [random.random() for _ in range(4)]
    s  = sum(xs)
    xs = [x/s for x in xs]
    return dict(zip(TARGETS, xs))


### 가중 카운터 & 워드클라우드 유틸

In [11]:
from wordcloud import WordCloud
import pandas as pd

# 누적 카운터(라벨별)
COUNTS = {lab: Counter() for lab in TARGETS}

def update_counts(text: str, scores: dict, use_weight=True):
    """
    use_weight=True  → 각 단어를 라벨 점수만큼 가중 합산 (ex. '우울증' 0.7이면 해당 단어에 +0.7)
    use_weight=False → argmax 라벨에만 +1 (전통적 방식)
    """
    labs = list(scores.keys())
    if use_weight:
        for tok in tokenize(text):
            if tok in STOPWORDS:
                continue
            for lab in TARGETS:
                COUNTS[lab][tok] += float(scores.get(lab, 0.0))
    else:
        lab = max(scores, key=scores.get)
        for tok in tokenize(text):
            if tok in STOPWORDS:
                continue
            COUNTS[lab][tok] += 1.0

def top_table(n=30):
    rows=[]
    for lab in TARGETS:
        for tok, cnt in COUNTS[lab].most_common(n):
            rows.append({"label": lab, "token": tok, "count": round(float(cnt),3)})
    return pd.DataFrame(rows)

def make_cloud(lab: str, width=1200, height=800):
    if not COUNTS[lab]:
        return None
    wc = WordCloud(
        font_path=FONT_PATH, width=width, height=height,
        background_color="white", max_words=200
    ).generate_from_frequencies(dict(COUNTS[lab]))
    return wc.to_image()


### Gradio 챗봇 UI

- 입력: 문장

- 모델 점수 자동 호출(또는 수동 점수 JSON로 덮어쓰기 가능)

- “가중합(soft)” vs “최대라벨( hard )” 모드 선택

- 4개 워드클라우드 + 상위 토큰 테이블 실시간 갱신

- 리셋/CSV 내보내기

In [12]:
import gradio as gr
import json
from io import BytesIO

def infer_and_update(message, mode, manual_scores_json):
    # 모델 점수 획득
    if manual_scores_json and manual_scores_json.strip():
        try:
            scores = json.loads(manual_scores_json)
        except Exception:
            return gr.update(), gr.update(), gr.update(), gr.update(), gr.update(value="JSON 파싱 실패"), None
    else:
        scores = get_scores(message)

    # 검증 & 정규화
    ssum = sum(float(scores.get(k,0.0)) for k in TARGETS)
    if ssum > 0:
        scores = {k: float(scores.get(k,0.0))/ssum for k in TARGETS}
    else:
        scores = {k: 0.0 for k in TARGETS}

    use_weight = (mode == "가중합(soft)")
    update_counts(message, scores, use_weight=use_weight)

    # 워드클라우드 생성
    imgs = [make_cloud(l) for l in TARGETS]
    # 상태 텍스트
    st = "scores: " + ", ".join([f"{k}:{scores[k]:.3f}" for k in TARGETS])
    return imgs[0], imgs[1], imgs[2], imgs[3], st, None

def export_counts():
    df = top_table(999999)
    csv = df.to_csv(index=False, encoding="utf-8-sig")
    return csv

def reset_all():
    for lab in TARGETS:
        COUNTS[lab].clear()
    return (None, None, None, None, "리셋 완료", None)

with gr.Blocks() as demo:
    gr.Markdown("### 단어 가중 카운터 · 워드클라우드 (우울/불안/중독/정상군)")
    with gr.Row():
        inp = gr.Textbox(label="문장 입력", placeholder="예: 요즘 불안하고 잠이 잘 안 와요…")
        mode = gr.Radio(choices=["가중합(soft)", "최대라벨(hard)"], value="가중합(soft)", label="집계 모드")
    manual = gr.Textbox(label="(선택) 점수 JSON 덮어쓰기", placeholder='예: {"우울증":0.7,"불안장애":0.2,"중독":0.05,"정상군":0.05}')

    submit = gr.Button("전송")
    reset  = gr.Button("리셋")
    export = gr.Button("CSV 내보내기")

    status = gr.Markdown("")
    with gr.Row():
        img_d = gr.Image(label="우울증",  type="pil")
        img_a = gr.Image(label="불안장애", type="pil")
    with gr.Row():
        img_s = gr.Image(label="중독",   type="pil")
        img_n = gr.Image(label="정상군", type="pil")

    table = gr.Dataframe(value=top_table(50), label="상위 토큰(라벨별)", interactive=False)

    def on_submit(message, mode, manual_json):
        img1, img2, img3, img4, st, _ = infer_and_update(message, mode, manual_json)
        # 표 갱신
        return img1, img2, img3, img4, st, top_table(50)

    submit.click(
        on_submit, inputs=[inp, mode, manual],
        outputs=[img_d, img_a, img_s, img_n, status, table]
    )

    reset.click(
        lambda: reset_all(),
        outputs=[img_d, img_a, img_s, img_n, status, table],
    )

    export.click(
        fn=lambda: export_counts(),
        outputs=gr.File(label="counts.csv"),
    )

demo.launch(debug=False)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c2b43667a695b46b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
